In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path



chrome_options = Options()
chrome_options.binary_location = "/Users/lilykoffman/Documents/athletics/chrome-mac-x64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    driver.get("https://worldathletics.org/results/olympic-games")
    time.sleep(3)  # wait for page to load

    # Find all <a> tags under the table with meeting names
    links = driver.find_elements(By.CSS_SELECTOR, "td[data-th='MEETING NAME'] a")

    games_urls = []
    for link in links:
        url = link.get_attribute("href")
        name = link.text
        print(f"{name}: {url}")
        games_urls.append(url)

finally:
    driver.quit()


The XXXIII Olympic Games: https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087
The XXXII Olympic Games (Athletics): https://worldathletics.org/results/olympic-games/2021/the-xxxii-olympic-games-athletics-6568
The XXXI Olympic Games: https://worldathletics.org/results/olympic-games/2016/the-xxxi-olympic-games-5771
The XXX Olympic Games: https://worldathletics.org/results/olympic-games/2012/the-xxx-olympic-games-4871
The XXIX Olympic Games: https://worldathletics.org/results/olympic-games/2008/the-xxix-olympic-games-3659
28th Olympic Games: https://worldathletics.org/results/olympic-games/2004/28th-olympic-games-3201
27th Olympic Games: https://worldathletics.org/results/olympic-games/2000/27th-olympic-games-2363


In [ ]:
def get_event_links(driver, games_url):
    print(f"Loading: {games_url}")
    driver.get(games_url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    all_event_links = []
    rows = soup.select("table.records-table tbody tr")

    for row in rows:
        cells = row.find_all("td")
        if not cells:
            continue

        # Use data-th attributes to be position-independent
        data = {cell.get("data-th", "").strip(): cell.get_text(strip=True) for cell in cells}

        sex = data.get("SEX", "")
        event_name = data.get("EVENT", "")
        round_name = data.get("ROUND", "")

        # Now handle link cells (label and href)
        for cell in cells:
            link_tag = cell.find("a")
            if link_tag and link_tag.get("href"):
                label = link_tag.text.strip()
                href = link_tag["href"]

                all_event_links.append({
                    "sex": sex,
                    "event": event_name,
                    "round": round_name,
                    "label": label,
                    "url": f"https://worldathletics.org{href}"
                })

    print(f"✓ Found {len(all_event_links)} event links")
    return all_event_links



In [3]:
driver = webdriver.Chrome(service=service, options=chrome_options)

all_events = []

for games_url in games_urls:
    try:
        events = get_event_links(driver, games_url)
        for event in events:
            event["games_url"] = games_url
            all_events.append(event)
        print(f"✓ Fetched {len(events)} events from: {games_url}")
    except Exception as e:
        print(f"✗ Error processing {games_url}: {e}")
    time.sleep(2)  # Politeness delay

print(f"\n✅ Total events collected: {len(all_events)}")

driver.quit()

Loading: https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087
✓ Found 391 event links (new structure)
✓ Fetched 391 events from: https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087
Loading: https://worldathletics.org/results/olympic-games/2021/the-xxxii-olympic-games-athletics-6568
✓ Found 357 event links (new structure)
✓ Fetched 357 events from: https://worldathletics.org/results/olympic-games/2021/the-xxxii-olympic-games-athletics-6568
Loading: https://worldathletics.org/results/olympic-games/2016/the-xxxi-olympic-games-5771
✓ Found 347 event links (new structure)
✓ Fetched 347 events from: https://worldathletics.org/results/olympic-games/2016/the-xxxi-olympic-games-5771
Loading: https://worldathletics.org/results/olympic-games/2012/the-xxx-olympic-games-4871
✓ Found 325 event links (new structure)
✓ Fetched 325 events from: https://worldathletics.org/results/olympic-games/2012/the-xxx-olympic-games-4871
Loading: https:/

In [ ]:
final_events = [e for e in all_events if "Final" == e.get("round", "")]

In [7]:

def scrape_results_table(driver, url, timeout=60):
    print(f"Loading results page: {url}")
    try:
        driver.get(url)

        # Wait until the table with class 'records-table' is present
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.records-table"))
        )
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.select_one("table.records-table")
        if not table:
            print("No table found on page.")
            return []

        headers = [th.get_text(strip=True) for th in table.select("thead th")]
        rows_data = []

        for row in table.select("tbody tr"):
            cells = row.find_all("td")
            if not cells or len(cells) != len(headers):
                continue
            row_dict = {headers[i]: cells[i].get_text(strip=True) for i in range(len(headers))}
            rows_data.append(row_dict)

        return rows_data

    except Exception as e:
        print(f"❌ Error loading results table from {url}: {e}")
        return []


In [8]:
driver = webdriver.Chrome(service=service, options=chrome_options)

for event in final_events:
    result_url = event["url"]  # Or whatever key you use
    results = scrape_results_table(driver, result_url)
    print(f"{len(results)} results scraped from {result_url}")
    event["results"] = results

driver.quit()

Loading results page: https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087/men/20-kilometres-race-walk/final/startlist#resultheader
49 results scraped from https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087/men/20-kilometres-race-walk/final/startlist#resultheader
Loading results page: https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087/men/20-kilometres-race-walk/final/result#resultheader
49 results scraped from https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087/men/20-kilometres-race-walk/final/result#resultheader
Loading results page: https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087/women/20-kilometres-race-walk/final/startlist#resultheader
45 results scraped from https://worldathletics.org/results/olympic-games/2024/the-xxxiii-olympic-games-7087/women/20-kilometres-race-walk/final/startlist#resultheader
Loading results pa

In [9]:

# Separate by label
startlist_entries = [e for e in final_events if e["label"] == "Startlist"]
result_entries = [e for e in final_events if e["label"] == "Result"]

In [ ]:
def expand_event_entries(entries):
    rows = []
    for event in entries:
        meta = {
            "sex": event["sex"],
            "event": event["event"],
            "round": event["round"],
            "label": event["label"],
            "url": event["url"],
            "games_url": event["games_url"]
        }
        for result_row in event["results"]:
            combined = {**meta, **result_row}
            rows.append(combined)
    return pd.DataFrame(rows)

# Convert to DataFrames
startlist_df = expand_event_entries(startlist_entries)
result_df = expand_event_entries(result_entries)


In [16]:
# Get the directory where the script is located
output_dir = Path.cwd().parent / "data" / "raw"
output_dir.mkdir(parents=True, exist_ok=True)


# Save the CSV
output_path = output_dir / "olympic_startlists.csv"
startlist_df.to_csv(output_path, index=False)

output_path = output_dir / "olympic_results.csv"
result_df.to_csv(output_path, index=False)

In [ ]:
# repeat for World Championships
service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    driver.get("https://worldathletics.org/results/world-athletics-championships")
    time.sleep(3)  # wait for page to load

    # Find all <a> tags under the table with meeting names
    links = driver.find_elements(By.CSS_SELECTOR, "td[data-th='MEETING NAME'] a")

    games_urls = []
    for link in links:
        url = link.get_attribute("href")
        name = link.text
        print(f"{name}: {url}")
        games_urls.append(url)

finally:
    driver.quit()


World Athletics Championships, Budapest 2023: https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606
World Athletics Championships, Oregon 2022: https://worldathletics.org/results/world-athletics-championships/2022/world-athletics-championships-oregon-2022-6266
IAAF World Athletics Championships, DOHA 2019: https://worldathletics.org/results/world-athletics-championships/2019/iaaf-world-athletics-championships-doha-2019-6033
IAAF World Championships London 2017: https://worldathletics.org/results/world-athletics-championships/2017/iaaf-world-championships-london-2017-5151
15th IAAF World Championships: https://worldathletics.org/results/world-athletics-championships/2015/15th-iaaf-world-championships-4875
14th IAAF World Championships: https://worldathletics.org/results/world-athletics-championships/2013/14th-iaaf-world-championships-4873
13th IAAF World Championships in Athletics: https://worldathletics.org/results/world-

In [18]:
driver = webdriver.Chrome(service=service, options=chrome_options)

all_events = []

for games_url in games_urls:
    try:
        events = get_event_links(driver, games_url)
        for event in events:
            event["games_url"] = games_url
            all_events.append(event)
        print(f"✓ Fetched {len(events)} events from: {games_url}")
    except Exception as e:
        print(f"✗ Error processing {games_url}: {e}")
    time.sleep(2)  # Politeness delay

print(f"\n✅ Total events collected: {len(all_events)}")

driver.quit()

Loading: https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606
✓ Found 357 event links (new structure)
✓ Fetched 357 events from: https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606
Loading: https://worldathletics.org/results/world-athletics-championships/2022/world-athletics-championships-oregon-2022-6266
✓ Found 356 event links (new structure)
✓ Fetched 356 events from: https://worldathletics.org/results/world-athletics-championships/2022/world-athletics-championships-oregon-2022-6266
Loading: https://worldathletics.org/results/world-athletics-championships/2019/iaaf-world-athletics-championships-doha-2019-6033
✓ Found 355 event links (new structure)
✓ Fetched 355 events from: https://worldathletics.org/results/world-athletics-championships/2019/iaaf-world-athletics-championships-doha-2019-6033
Loading: https://worldathletics.org/results/world-athletics-

In [ ]:
final_events = [e for e in all_events if "Final" == e.get("round", "")]

[{'sex': 'M',
  'event': '20 Kilometres Race Walk',
  'round': 'Final',
  'label': 'Startlist',
  'url': 'https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/20-kilometres-race-walk/final/startlist#resultheader',
  'games_url': 'https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606'},
 {'sex': 'M',
  'event': '20 Kilometres Race Walk',
  'round': 'Final',
  'label': 'Result',
  'url': 'https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/20-kilometres-race-walk/final/result#resultheader',
  'games_url': 'https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606'},
 {'sex': 'M',
  'event': 'Shot Put',
  'round': 'Final',
  'label': 'Startlist',
  'url': 'https://worldathletics.org/results/world-athletics-championships/2023/w

In [21]:
driver = webdriver.Chrome(service=service, options=chrome_options)

for event in final_events:
    result_url = event["url"]  # Or whatever key you use
    results = scrape_results_table(driver, result_url)
    print(f"{len(results)} results scraped from {result_url}")
    event["results"] = results

driver.quit()

Loading results page: https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/20-kilometres-race-walk/final/startlist#resultheader
50 results scraped from https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/20-kilometres-race-walk/final/startlist#resultheader
Loading results page: https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/20-kilometres-race-walk/final/result#resultheader
50 results scraped from https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/20-kilometres-race-walk/final/result#resultheader
Loading results page: https://worldathletics.org/results/world-athletics-championships/2023/world-athletics-championships-budapest-2023-6606/men/shot-put/final/startlist#resultheader
12 results scraped from htt

In [22]:
startlist_entries = [e for e in final_events if e["label"] == "Startlist"]
result_entries = [e for e in final_events if e["label"] == "Result"]
startlist_df = expand_event_entries(startlist_entries)
result_df = expand_event_entries(result_entries)

In [23]:
# Save the CSV
output_path = output_dir / "wch_startlists.csv"
startlist_df.to_csv(output_path, index=False)

output_path = output_dir / "wch_results.csv"
result_df.to_csv(output_path, index=False)